In [ ]:
import os
import re
import json
import pdfplumber
import openai
import getpass
from langchain.text_splitter import RecursiveCharacterTextSplitter
from docx import Document
import time

# --- API Key Setup (unchanged) ---
if "DEEPSEEK_API_KEY" not in os.environ:
    os.environ["DEEPSEEK_API_KEY"] = getpass.getpass("Enter your Deepseek API Key: ")

# --- PDF Extraction (unchanged) ---
def extract_pdf_text(pdf_path):
    full_text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text = page.extract_text() or ""
                full_text += text + "\n"
        return full_text.strip()
    except Exception as e:
        print(f"⚠️ 无法打开或处理 PDF 文件 {pdf_path}: {e}")
        return None

# --- Text Splitting (Updated Defaults) ---
def split_text(text, chunk_size=25000, overlap=500): # Updated defaults
    if not text:
        return []
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=overlap,
        separators=["\n\n", "\n", ". ", "! ", "? ", " ", ""], # Added spaces after punctuation
        length_function=len,
        is_separator_regex=False,
    )
    return splitter.split_text(text)

# --- Deepseek Processing (Updated max_tokens) ---
def gpt_process_text(text_chunk):
    # --- Prompt (unchanged) ---
    prompt = f"""
    You are an AI assistant responsible for extracting sentences relevant to Ultra-High Performance Concrete (UHPC) from research papers. Your task is to identify, clean, and extract relevant sentences while preserving their original structure.

    **Sentence Selection and Cleaning Criteria:**
    1️⃣ Extract sentences that explicitly mention 'UHPC' or 'Ultra-High Performance Concrete' (case-insensitive).
    2️⃣ Extract sentences that discuss UHPC-related topics even if they do not contain 'UHPC'.
    3️⃣ Sentences should be relevant, not generic. Ignore vague statements like “Concrete has high strength.”
    4️⃣ Sentences should be at least 8 words long to ensure meaningful content.
    5️⃣ Exclude references, captions, formulas, and table data to prevent noisy or incomplete extractions.
    6️⃣ Do not modify, rewrite, or summarize extracted sentences. Preserve their original wording, structure, and punctuation.
    7️⃣ If a sentence contains references like [1] or (2023), remove only the reference markers, keeping the rest of the sentence intact.
    Return the cleaned and validated sentences as a JSON list. Ensure the JSON format is strictly adhered to.

    **✅ Output Format (Strict JSON Only):**
    ```json
    {{
        "valid_sentences": ["sentence1", "sentence2", "sentence3"]
    }}
    ```
    """
    try:
        client = openai.OpenAI(
            api_key=os.environ["DEEPSEEK_API_KEY"],
            base_url="https://api.deepseek.com/v1"
        )
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": text_chunk}
            ],
            temperature=0.3,
            max_tokens=7800, # Updated max_tokens
            response_format={"type": "json_object"}
        )
        gpt_output = response.choices[0].message.content.strip()

        # --- JSON Parsing Logic (improved robustness slightly) ---
        def parse_gpt_response(response_text):
            try:
                # Attempt direct JSON parsing first, as response_format should ensure it
                return json.loads(response_text)
            except json.JSONDecodeError as e:
                 print(f"⚠️ 初步 JSON 解析错误: {e}")
                 print(f"   尝试清理和解析...")
                 try:
                    # Fallback: Clean potential markdown and leading/trailing garbage
                    json_match = re.search(r'\{.*\}', response_text, re.DOTALL)
                    if json_match:
                        cleaned_text = json_match.group(0)
                        result = json.loads(cleaned_text)
                        print("   通过正则清理后解析成功。")
                        return result
                    else:
                        print(f"   无法在响应中找到有效的 JSON 结构: {response_text[:500]}...")
                        return None
                 except json.JSONDecodeError as e2:
                    print(f"   清理后 JSON 解析仍然失败: {e2}")
                    print(f"   收到内容 (前500字符): {response_text[:500]}...")
                    return None # Give up if parsing fails

        result = parse_gpt_response(gpt_output)
        return result.get("valid_sentences", []) if result else []

    # --- Error Handling (unchanged, but RateLimitError sleep is important) ---
    except openai.APIConnectionError as e:
        print(f"❌ Deepseek API 连接失败: {e}")
        return []
    except openai.RateLimitError as e:
        print(f"⏳ Deepseek API 速率限制，等待 15 秒后重试...") # Increased wait time slightly
        time.sleep(15)
        return [] # Simplistic handling: skip chunk on rate limit for now
    except openai.APIStatusError as e:
        # Check for specific error codes, like context length exceeded (though unlikely with current chunk size)
        # or output token limit exceeded
        print(f"❌ Deepseek API 状态错误: {e.status_code}")
        print(f"   响应: {e.response}")
        # Potentially log e.response.text to see the error message from Deepseek
        if "maximum context length" in str(e.response).lower():
             print("   错误提示：可能超过了最大上下文长度！")
        if "maximum number of tokens" in str(e.response).lower():
             print("   错误提示：可能超过了最大输出 token 数！输出可能被截断。")
        return []
    except Exception as e:
        print(f"❌ Deepseek 处理失败: {e}")
        return []

# --- Saving to Doc (unchanged) ---
def save_sentences_to_doc(sentences, doc_path):
    try:
        doc = Document()
        doc.add_heading("Step1_Text Generation (DeepSeek - Optimized)", level=1)
        count = 0
        if isinstance(sentences, list): # Ensure it's a list
            for i, sentence in enumerate(sentences, 1):
                if isinstance(sentence, str):
                    doc.add_paragraph(f"{i}. {sentence}")
                    count += 1
                else:
                    print(f"⚠️ 跳过非字符串项目索引 {i}: {sentence}")
            doc.save(doc_path)
            print(f"✅ 已成功保存 {count} 个句子至 {doc_path}")
        else:
             print(f"❌ 预期句子列表，但收到: {type(sentences)}")
    except Exception as e:
        print(f"❌ 保存 Word 文档失败: {e}")

# --- Main Program Logic (unchanged structure, uses updated defaults) ---
def main(pdf_paths, output_doc_path):
    all_relevant_sentences = []
    total_files = len(pdf_paths)

    for idx, pdf_path in enumerate(pdf_paths):
        print(f"\n🚀 处理文件 {idx+1}/{total_files}: {os.path.basename(pdf_path)}")
        if not os.path.exists(pdf_path):
            print(f"⚠️ 文件未找到，跳过: {pdf_path}")
            continue

        try:
            text = extract_pdf_text(pdf_path)
            if not text:
                print(f"⚠️ 未能从 {os.path.basename(pdf_path)} 提取文本或文件为空，跳过。")
                continue

            print("📖 正在拆分文本 (chunk_size=25000, overlap=500)...")
            # Uses the new defaults from split_text definition
            text_chunks = split_text(text)
            print(f"🔹 文本分为 {len(text_chunks)} 个部分。")

            if not text_chunks:
                 print(f"⚠️ 文本拆分后为空，跳过文件 {os.path.basename(pdf_path)}。")
                 continue

            chunk_sentences = []
            for i, chunk in enumerate(text_chunks):
                # Add check for chunk size just in case splitter produces large chunks
                if len(chunk) > 30000: # Arbitrary safety check slightly above target
                     print(f"  ⚠️ 块 {i+1} 字符数异常 ({len(chunk)})，可能导致问题。跳过此块。")
                     continue

                print(f"🤖 使用 Deepseek 处理第 {i+1}/{len(text_chunks)} 段文本 (max_tokens=7800)...")
                sentences = gpt_process_text(chunk)
                if sentences:
                     chunk_sentences.extend(sentences)
                time.sleep(1.5) # Slightly increase sleep time due to larger requests

            print(f"📄 文件 {os.path.basename(pdf_path)} 处理完毕，找到 {len(chunk_sentences)} 个相关句子。")
            all_relevant_sentences.extend(chunk_sentences)

        except Exception as e:
            print(f"❌ 处理文件 {os.path.basename(pdf_path)} 时发生意外错误: {e}")
            continue

    print(f"\n✅ 所有文件处理完毕，总筛选后的句子数量: {len(all_relevant_sentences)}")

    if all_relevant_sentences:
        print("📑 保存结果到文档...")
        output_dir = os.path.dirname(output_doc_path)
        try:
            if output_dir and not os.path.exists(output_dir):
                 os.makedirs(output_dir)
                 print(f"   创建输出目录: {output_dir}")
            save_sentences_to_doc(all_relevant_sentences, output_doc_path)
        except Exception as e:
             print(f"❌ 创建输出目录或保存文件时出错: {e}")
    else:
        print("⏹️ 未提取到相关句子，不生成输出文件。")


# --- Execution Block (Path generation unchanged) ---
if __name__ == "__main__":
    pdf_base_dir = r"D:\SIT\knowledge\data"
    num_files = 50
    if not os.path.isdir(pdf_base_dir):
        print(f"错误：指定的 PDF 基础目录不存在: {pdf_base_dir}")
        exit()
    pdf_files = [os.path.join(pdf_base_dir, f"{i}.pdf") for i in range(1, num_files + 1)]

    output_doc = r"D:\SIT\knowledge\Result\Step1_UHPC_Sentences_Extracted_DeepSeek_Optimized.docx" # Updated output name

    print(f"将处理 {len(pdf_files)} 个 PDF 文件从目录: {pdf_base_dir}")
    print(f"使用设置: chunk_size=25000, overlap=500, max_tokens=7800")
    print(f"结果将保存至: {output_doc}")

    main(pdf_files, output_doc)